## Простейшая модель 

### План

- преобразовать текст в ветор частей речи (+ падеж + спряжение там где это актуально)
- построить модель на би и триграммах


### Загрузка данных

In [1]:
import boto3
import pandas as pd
import re
import nltk

from pymystem3 import Mystem
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from nltk import ngrams
from sklearn.linear_model import LogisticRegression # можно заменить на любимый классификатор
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer


In [2]:
session = boto3.session.Session()

s3 = session.client(
    service_name='s3',
    endpoint_url='https://storage.yandexcloud.net',
    aws_access_key_id='REDACTED_KEY_ID_1',
    aws_secret_access_key='REDACTED_SECRET_KEY_1',
    region_name='ru-cental1'
)

BUCKET_NAME = "mlds23-authorship-identification"
BUCKET_DIR = "splitted_data/"
FILENAME = 'splitted_df.csv'
#загружает в локальную директорию, потом отдельно надо считывать
#s3.download_file(Filename="data/" + FILENAME, Bucket=BUCKET_NAME, Key=BUCKET_DIR + FILENAME)

In [3]:
df = pd.read_csv('splitted_df.csv')
print(df.shape)
df.head()

(11715, 3)


,target,text,book
0,author_id_06,"Узкими горными тропинками , от одного дачного ...",raw_data/aleksandr_kuprin_belyj_pudel'.txt
1,author_id_06,свернутый ковер для акробатических упражнений ...,raw_data/aleksandr_kuprin_belyj_pudel'.txt
2,author_id_06,"позабытые . Кроме того , были в шарманке две п...",raw_data/aleksandr_kuprin_belyj_pudel'.txt
3,author_id_06,тайной грусти : — Что поделаешь ? .. Древний о...,raw_data/aleksandr_kuprin_belyj_pudel'.txt
4,author_id_06,", да уж ладно ! Кормила она нас с тобой , Серг...",raw_data/aleksandr_kuprin_belyj_pudel'.txt


In [4]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['target'], test_size=.3, random_state=42)


## Кодирование признаков

- исходим из предположения что знаки препинания являются важными признаками, поэтому закодируем некоторые из них в слова
- закодируем слова как часть речи, для частей речи имеющих падеж добавим падеж (сущ, прил, местоимение), для частей речи имеющих спряжение добавим спряжение. Формат - ЧАСТЬРЕЧИ_ПАДЕЖ 

In [5]:
def clean_sentence(sentence):
    sentence = re.sub(r"[^а-яА-ЯёЁ \-\"!'(),.:;?]", '', sentence) 
    return sentence

# тест
clean_sentence("как-то - 'рано' 'Marta'? пела: лЕСОМ, * &нифига(она) не ела")

"как-то - 'рано' ''? пела: лЕСОМ,  нифига(она) не ела"

In [6]:
def make_punkt(sentence):
    repl = [('.',' POINT '), (',',' COMMA '), ('?',' QST '), ('!',' EXCL '), 
            (':',' COLON '), (';',' SEMICOL '), (',',' DASH '),]
    for p, r in repl:
        sentence = sentence.replace(p,r)
    sentence = re.sub(r"\s?-\s|\s-\s?", ' DASH ', sentence) # не трогать тире в слове (как-то)
        
    return sentence
        
# тест    
make_punkt("как-то - 'рано' 'Marta'? пела: лЕСОМ, * &нифига(она) не ела")    

"как-то DASH 'рано' 'Marta' QST  пела COLON  лЕСОМ COMMA  * &нифига(она) не ела"

In [7]:
mystem_analyzer = Mystem()

def make_grams(sentence):
    morph = mystem_analyzer.analyze(sentence)
    
    ret = []
    for lex in morph:
        if lex['text'] in ['POINT', 'COMMA', 'QST', 'EXCL', 'COLON', 'SEMICOL', 'DASH']:
            ret.append(lex['text'])
            continue
        
        try:
            if 'analysis' in lex.keys() and 'gr' in lex['analysis'][0].keys():
                ret.append(lex['analysis'][0]['gr'])
        except:
            # встретил что-то непотребное в стиле ру-ру-ру
            pass
    return ' '.join(ret)

# тест
make_grams("и POINT столу но и тёплым снегом")

'CONJ= POINT S,муж,неод=дат,ед CONJ= CONJ= A=(дат,мн,полн|твор,ед,полн,муж|твор,ед,полн,сред) S,муж,неод=твор,ед'

In [8]:
def make_grams_brief(sentence):
    morph = mystem_analyzer.analyze(sentence)
    
    ret = []
    for lex in morph:
        if lex['text'] in ['POINT', 'COMMA', 'QST', 'EXCL', 'COLON', 'SEMICOL', 'DASH']:
            ret.append(lex['text'])
            continue
        
        try:
            if 'analysis' in lex.keys() and 'gr' in lex['analysis'][0].keys():
                ret.append(lex['analysis'][0]['gr'].split('=')[0])
        except:
            # встретил что-то непотребное в стиле ру-ру-ру
            pass
    return ' '.join(ret)

# тест
make_grams_brief("и POINT столу но и тёплым снегом")

'CONJ POINT S,муж,неод CONJ CONJ A S,муж,неод'

In [9]:
def prepare_text(Text_corp):
    res = []
    for text in Text_corp:
        text = clean_sentence(text)
        text = make_punkt(text)
        text = make_grams(text)
        res.append(text)
    return res

X_tr = prepare_text(X_train)
X_ts = prepare_text(X_test)


In [10]:
type(X_tr)

list

## Подготовка BOW и обучение простейшей линейной модели



In [11]:
myvec = CountVectorizer(ngram_range=(1, 3), 
                        stop_words=None, lowercase=False, 
                        token_pattern=r"(?u)\b[a-zA-Zа-яА-ЯёЁ0-9,=90|\-]+\b")
X_tr_bow  = myvec.fit_transform(X_tr)
X_ts_bow  = myvec.transform(X_ts)
#myvec.vocabulary_.items()

In [12]:
clf = LogisticRegression(random_state=42, max_iter=1000)
clf.fit(X_tr_bow, y_train)

LogisticRegression(max_iter=1000, random_state=42)

In [13]:
pred = clf.predict(X_ts_bow)
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

author_id_00       0.71      0.68      0.70       419
author_id_01       0.69      0.70      0.70       110
author_id_02       0.73      0.65      0.69       906
author_id_03       0.44      0.57      0.50       192
author_id_04       0.60      0.60      0.60       373
author_id_05       0.56      0.62      0.59       192
author_id_06       0.52      0.53      0.52       308
author_id_07       0.50      0.65      0.57        99
author_id_08       0.31      0.54      0.40       115
author_id_09       0.75      0.68      0.71       801

    accuracy                           0.64      3515
   macro avg       0.58      0.62      0.60      3515
weighted avg       0.66      0.64      0.64      3515



### Обучимся по схеме TF-IDF

In [14]:
vec = TfidfVectorizer(ngram_range=(2, 3))
X_tr_tfidf = vec.fit_transform(X_tr)
X_ts_tfidf = vec.transform(X_ts)

clf = LogisticRegression(random_state=42,  max_iter=1000)
clf.fit(X_tr_tfidf, y_train)
pred = clf.predict(X_ts_tfidf)

print(classification_report(pred, y_test))

              precision    recall  f1-score   support

author_id_00       0.65      0.61      0.63       431
author_id_01       0.32      0.95      0.47        37
author_id_02       0.76      0.52      0.62      1159
author_id_03       0.25      0.58      0.35       110
author_id_04       0.51      0.54      0.52       353
author_id_05       0.35      0.55      0.43       136
author_id_06       0.42      0.48      0.45       272
author_id_07       0.20      0.86      0.32        29
author_id_08       0.13      0.54      0.21        48
author_id_09       0.76      0.58      0.66       940

    accuracy                           0.56      3515
   macro avg       0.43      0.62      0.47      3515
weighted avg       0.64      0.56      0.58      3515



### Попробуем улучшить результат сократив информацию по частям речи

In [15]:
def prepare_text(Text_corp):
    res = []
    for text in Text_corp:
        text = clean_sentence(text)
        text = make_punkt(text)
        text = make_grams_brief(text)
        res.append(text)
    return res

X_trb = prepare_text(X_train)
X_tsb = prepare_text(X_test)


In [16]:
myvec = CountVectorizer(ngram_range=(1, 4), 
                        stop_words=None, lowercase=False, 
                        token_pattern=r"(?u)\b[a-zA-Zа-яА-ЯёЁ0-9,=90|\-]+\b")
X_trb_bow  = myvec.fit_transform(X_trb)
X_tsb_bow  = myvec.transform(X_tsb)
#myvec.vocabulary_.items()

In [20]:
clf = LogisticRegression(random_state=42, max_iter=1500)
clf.fit(X_trb_bow, y_train)

LogisticRegression(max_iter=1500, random_state=42)

In [21]:
pred = clf.predict(X_tsb_bow)
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

author_id_00       0.67      0.66      0.67       408
author_id_01       0.56      0.62      0.59       100
author_id_02       0.70      0.63      0.66       901
author_id_03       0.39      0.53      0.45       187
author_id_04       0.55      0.56      0.55       367
author_id_05       0.53      0.58      0.55       197
author_id_06       0.54      0.52      0.53       320
author_id_07       0.51      0.61      0.56       107
author_id_08       0.29      0.50      0.37       115
author_id_09       0.74      0.65      0.69       813

    accuracy                           0.61      3515
   macro avg       0.55      0.59      0.56      3515
weighted avg       0.63      0.61      0.61      3515



### Тоже с использованием TF-IDF

In [19]:
vec = TfidfVectorizer(ngram_range=(2, 3))
X_trb_tfidf = vec.fit_transform(X_trb)
X_tsb_tfidf = vec.transform(X_tsb)

clf = LogisticRegression(random_state=42,  max_iter=1000)
clf.fit(X_trb_tfidf, y_train)
pred = clf.predict(X_tsb_tfidf)

print(classification_report(pred, y_test))

              precision    recall  f1-score   support

author_id_00       0.61      0.61      0.61       406
author_id_01       0.41      0.87      0.55        52
author_id_02       0.75      0.53      0.62      1132
author_id_03       0.27      0.52      0.36       130
author_id_04       0.50      0.51      0.50       374
author_id_05       0.43      0.60      0.50       155
author_id_06       0.40      0.48      0.43       256
author_id_07       0.29      0.73      0.42        51
author_id_08       0.17      0.51      0.25        65
author_id_09       0.74      0.59      0.66       894

    accuracy                           0.56      3515
   macro avg       0.46      0.59      0.49      3515
weighted avg       0.62      0.56      0.58      3515

